<a href="https://colab.research.google.com/github/prasgane/SpeedChallenge/blob/sonnet/how_to_win_the_speed_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import glob
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
from tensorflow.keras.layers import TimeDistributed, Input, Flatten, Dense, LSTM, Dropout, Conv2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
import sonnet as snt

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.direct = "./drive/My Drive/commai_dataset/"

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        
        # Find list of IDs
        list_IDs_temp = self.list_IDs[index]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.load(self.direct+"data/data_" + list_IDs_temp +".npy")
        y = np.load(self.direct+"labels/label_" + list_IDs_temp +".npy")
        
        return X, y

In [ ]:
def sum_sq_err(y_true, y_pred):
    return tf.keras.backend.sum(tf.keras.backend.square(y_true - y_pred))

In [ ]:
params = {'dim': (110,320),
          'batch_size': 101,
          'n_classes': 1,
          'n_channels': 3,
          'shuffle': False}

train_data = []
train_label = []
valid_data = []
valid_label = []

for i in range(70):
    train_data.append("%03d" %i)
    train_data.append("%03d" %(i+100))

for i in range(70, 100):
    valid_data.append("%03d" %i)
    valid_data.append("%03d" %(i+100))

partition={'train':train_data, 'validation':valid_data}
labels = {'train': train_label,'validation':valid_label}

training_generator = DataGenerator(partition['train'], labels['train'], **params)
validation_generator = DataGenerator(partition['train'], labels['train'], **params)

In [ ]:
# def resnet_model():
resnet = ResNet50(include_top=False, weights='imagenet')
input_test = Input(shape=(None,110,320,3))
x = TimeDistributed(resnet)(input_test)
x = TimeDistributed(Flatten())(x)
x = TimeDistributed(Dropout(0.2))(x)
# x = LSTM(256,return_sequences=True)(x)
# x = TimeDistributed(Dropout(0.2))(x)
x = LSTM(256,return_sequences=True)(x)
x = Dense(128)(x)
output_layer = Dense(1)(x)
model = tf.keras.models.Model(inputs=[input_test], outputs=[output_layer])


In [ ]:
# def get_rico_model:
model = Sequential()
model.add(TimeDistributed(Conv2D(4, (32, 32), activation='relu', kernel_initializer='he_uniform', padding='same'), input_shape=(None,110, 320, 3)))
model.add(TimeDistributed(Conv2D(16, (16, 16), activation='relu', kernel_initializer='he_uniform', padding='same')))
model.add(TimeDistributed(Conv2D(32, (8, 8), activation='relu', kernel_initializer='he_uniform', padding='same')))
model.add(TimeDistributed(Conv2D(64, (4, 4), activation='relu', kernel_initializer='he_uniform', padding='same')))
# model.add(TimeDistributed(Dense(512)))
model.add(TimeDistributed(Flatten()))
# model.add(LSTM(512,return_sequences=True))
model.add(TimeDistributed(Dense(1)))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, None, 110, 320, 4) 12292     
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 110, 320, 16 16400     
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 110, 320, 32 32800     
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 110, 320, 64 32832     
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 2252800)     0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 1)           2252801   
Total params: 2,347,125
Trainable params: 2,347,125
Non-trainable params: 0
______________________________________________

In [ ]:
_optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=_optimizer, loss = sum_sq_err)

In [ ]:
model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=5)

Epoch 1/5


ValueError: ignored